# Instruction Synthesizer Technique

In [1]:
!pip install "distilabel[hf-transformers, openai]>=1.0.0"


[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
from distilabel.pipeline import Pipeline
from distilabel.steps import LoadDataFromHub, KeepColumns, LoadDataFromDicts
from distilabel.steps.tasks import Genstruct
from distilabel.llms import TransformersLLM
from distilabel.steps import Step, StepInput
from distilabel.steps.typing import StepOutput
from typing import List
from pydantic import Field, PrivateAttr
from transformers import AutoModelForCausalLM, AutoTokenizer

In [3]:

def parse_pred(pred):
    """Extract the list of instruction-response pairs from the prediction"""
    QA_str_list = pred.split('</END>')
    if not pred.endswith('</END>'):
        QA_str_list = QA_str_list[:-1]

    QA_list = []
    raw_questions = []
    for QA_str in QA_str_list:
        try:
            assert len(QA_str.split('<ANS>')) == 2, f'invalid QA string: {QA_str}'
            Q_str, A_str = QA_str.split('<ANS>')
            Q_str, A_str = Q_str.strip(), A_str.strip()
            assert Q_str.startswith('<QUE>'), f'invalid question string: {Q_str} in QA_str: {QA_str}'
            assert len(A_str) > 0, f'invalid answer string in QA_str: {QA_str}'
            Q_str = Q_str.replace('<QUE>', '').strip()
            assert Q_str.lower() not in raw_questions, f'duplicate question: {Q_str}'
            QA_list.append({'Q': Q_str, 'A': A_str})
            raw_questions.append(Q_str.lower())
        except:
            pass

    return QA_list

def get_instruction_response_pairs(context, model, tokenizer):
    '''Prompt the synthesizer to generate instruction-response pairs based on the given context'''
    prompt = f'<s> <CON> {context} </CON>\n\n'
    inputs = tokenizer(prompt, add_special_tokens=False, return_tensors="pt").input_ids.to(model.device)
    outputs = model.generate(input_ids=inputs, max_new_tokens=400, do_sample=False)[0]

    pred_start = int(inputs.shape[-1])
    pred = tokenizer.decode(outputs[pred_start:], skip_special_tokens=True)
    return parse_pred(pred)

In [4]:
# # Get the generated instruction-response paris
# instruction_response_pairs = get_instruction_response_pairs(context)

# # Print out the results
# print(f'# Context:\n{context}\n')
# for index, pair in enumerate(instruction_response_pairs):
#     print(f'## Instruction {index + 1}:\n{pair["Q"]}\n## Response {index + 1}:\n{pair["A"]}\n')

In [15]:
from transformers.models.mistral.modeling_mistral import MistralForCausalLM
from transformers.models.llama.tokenization_llama_fast import LlamaTokenizerFast

class InstructionSynthesizer(Step):

    def __init__(self, name: str):
        super().__init__(name=name)
        self._model = None
        self._tokenizer = None
    
    @property
    def inputs(self) -> List[str]:
        # Specify the input fields expected by this step
        return ['text']

    @property
    def outputs(self) -> List[str]:
        # Specify the output fields that this step will produce
        return ['instruction', 'response']

    def process(self, inputs: StepInput) -> StepOutput:
        if self._model is None:
            self._model = AutoModelForCausalLM.from_pretrained("instruction-pretrain/instruction-synthesizer").to('cuda')
            self._tokenizer = AutoTokenizer.from_pretrained("instruction-pretrain/instruction-synthesizer")
        results = []
        for example in inputs:
            context = example['text']
            instruction_response_pairs = get_instruction_response_pairs(context, self._model, self._tokenizer)
            print(instruction_response_pairs)
            text_index = inputs.index(example)
            for pair in instruction_response_pairs:
                results.append({"text": context, "instruction": pair["Q"], "response": pair["A"]})
        yield results

In [18]:
with Pipeline(name='synthesizer-pipeline') as pipeline:
    load_data_from_hub = LoadDataFromHub(
        name='load-data-from-hub',
        # output_mappings={'prompt': 'text'}
    )

    synthesizer = InstructionSynthesizer(
        name='synthesizer',
        input_batch_size = 1
    )
    
    load_data_from_hub >> synthesizer 


In [19]:
distiset = pipeline.run(
    parameters={
        load_data_from_hub.name: {
            'repo_id': "ahsanirfan961/title-content-dataset",
            "split": "train"
        },
    },
)

README.md:   0%|          | 0.00/299 [00:00<?, ?B/s]

[09/11/24 14:57:25] INFO     ['distilabel.pipeline'] 📝 Pipeline data will be written to                ]8;id=636869;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/base.py\base.py]8;;\:]8;id=876734;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/base.py#696\696]8;;\
                             '/root/.cache/distilabel/pipelines/synthesizer-pipeline/2933172e5507b09605            
                             b227042779cd3e4cebf456/data'                                                          

                    INFO     ['distilabel.pipeline'] ⌛ The steps of the pipeline will be loaded in     ]8;id=421415;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/base.py\base.py]8;;\:]8;id=747324;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/base.py#705\705]8;;\
                             stages:                                                                               
                              * Stage 0: ['load-data-from-hub', 'synthesizer']                                     

                    INFO     ['distilabel.pipeline'] ⏳ Waiting for all the steps of stage 0 to load... ]8;id=603318;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/base.py\base.py]8;;\:]8;id=570833;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/base.py#918\918]8;;\

[09/11/24 14:57:28] INFO     ['distilabel.pipeline'] ⏳ Steps from stage 0 loaded: 2/2                  ]8;id=77972;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/base.py\base.py]8;;\:]8;id=159206;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/base.py#950\950]8;;\
                              * 'load-data-from-hub' replicas: 1/1                                                 
                              * 'synthesizer' replicas: 1/1                                                        

...

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[{'Q': 'who called the move a dangerous escalation?', 'A': "qatar's foreign ministry"}, {'Q': 'what did sheikh mohammed bin abdulrahman althani call the move?', 'A': 'a dangerous escalation'}]


[09/11/24 14:59:22] INFO     ['distilabel.step.synthesizer'] 📨 Step 'synthesizer' sending      ]8;id=210973;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/step_wrapper.py\step_wrapper.py]8;;\:]8;id=686768;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/step_wrapper.py#274\274]8;;\
                             batch 0 to output queue                                                               

                    INFO     ['distilabel.step.synthesizer'] 🏁 Finished running step           ]8;id=908863;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/step_wrapper.py\step_wrapper.py]8;;\:]8;id=450181;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/step_wrapper.py#127\127]8;;\
                             'synthesizer' (replica ID: 0)                                                         

Generating train split: 0 examples [00:00, ? examples/s]

In [20]:
print(distiset)

Distiset({
    default: DatasetDict({
        train: Dataset({
            features: ['text', 'instruction', 'response'],
            num_rows: 46
        })
    })
})


In [21]:
print(distiset['default']['train'].to_pandas())

                                                 text  \
0   a us navy transport plane carrying  people cra...   
1   a us navy transport plane carrying  people cra...   
2   a us navy transport plane carrying  people cra...   
3   a us navy transport plane carrying  people cra...   
4   a us navy transport plane carrying  people cra...   
5   a us navy transport plane carrying  people cra...   
6   a us navy transport plane carrying  people cra...   
7   a us navy transport plane carrying  people cra...   
8   zimbabwe s former vice president emmerson mnan...   
9   the mayor and police chief of cleveland next w...   
10  the mayor and police chief of cleveland next w...   
11  the mayor and police chief of cleveland next w...   
12  the mayor and police chief of cleveland next w...   
13  the mayor and police chief of cleveland next w...   
14  the mayor and police chief of cleveland next w...   
15  the mayor and police chief of cleveland next w...   
16  the mayor and police chief 

In [13]:
print(distiset['default']['train'].to_pandas()['instruction'][0])

What type of energy does the author likely advocate for?
Options:
- wind energy
- solar energy
- no energy
- hydroelectric energy
- fossil fuels
- kinetic energy
- nuclear energy
- coal energy
Let's think step by step.


In [14]:
print(distiset['default']['train'].to_pandas()['response'][0])

Renewable energy sources such as solar, wind, and hydroelectric power are becoming increasingly important in the fight against climate change. These energy sources produce little to no greenhouse gas emissions, making them environmentally friendly alternatives to fossil fuels.
Therefore, the answer is solar energy
